In [2]:
import numpy as np
import torch

---

### Review of Pt. 1

In [2]:
def f(x: np.ndarray):
    return np.exp(x)

If $ \bm{x} = [x_1, x_2, x_3] $, then $ f(\bm{x}) = [x_1^e, x_2^e, x_3^e] $.

$ f(\bm{x}): R^3 \rightarrow R^3 $, thus the Jacobian should be of shape `3 x 3`.

The Jacobian of $ f(\bm{x}) $, $J(f)$, is the matrix:

$$\begin{bmatrix}

{\frac{\partial f_1}{x_1}} & {\frac{\partial f_1}{x_2}} & {\frac{\partial f_1}{x_3}} \\
{\frac{\partial f_2}{x_1}} & {\frac{\partial f_2}{x_2}} & {\frac{\partial f_2}{x_3}} \\
{\frac{\partial f_3}{x_1}} & {\frac{\partial f_3}{x_2}} & {\frac{\partial f_3}{x_3}}

\end{bmatrix}$$

$$=\begin{bmatrix}
{x_1^e} & {0} & {0} \\
{0} & {x_2^e} & {0} \\
{0} & {0} & {x_3^e}
\end{bmatrix}$$

Like for *most* other functions, $J(f)$ represents an affine, diagonal linear mapping.

In [3]:
def J_f(x: np.ndarray):
    return np.eye(x.shape[0]) * np.exp(x)

Now, what does the **VJP** (vector-Jacobian product) really *mean*?

We want to find a sensitivity "mapping" to see how each of the outputs are affected by each element of the input vector $\bm{x}$.

To do this, we left multiply the transpose of our desired "weighting" (see Pt. 1) by $J(f)$.

In [4]:
x = np.array([1, 2, 3])
f_x = f(x)
f_x

array([ 2.71828183,  7.3890561 , 20.08553692])

In [5]:
u = np.array([[1],
              [1],
              [1]])
u.T @ J_f(x)

array([[ 2.71828183,  7.3890561 , 20.08553692]])

The VJP is exactly the same as $f(\bm{x})$, which is what we'd expect.

Why?

- We want to find the sensitivity of $f(x_1), f(x_2), f(x_3)$ to $x_1, x_2, x_3$.
- By setting `u` to a `1 x 3` ones-vector, we're saying that we want to see the sensitivity of all 3 outputs to the 3 inputs.
    - Think of `u` as a "masking" or "weighting" vector for each element's sensitivity.
- Intuitively, if $f(x_1) = x_1^e$, then any perturbation $h$ in $x_1$ will directly propagate to $f(x_1)$.
    - $(x_1 + h)^e = f(x_1 + h)$.

To compare w/ PyTorch...

In [6]:
x = torch.rand(1, 3, requires_grad=True)
y = x.exp()

In [7]:
y.backward(torch.ones_like(x))
assert (y == x.grad).all()

(This is all in an ideal situation. In reality, it's inefficient to find Jacobians for *every* operation and compute VJPs explicitly.)

---

### Gradient of element-wise operations

Suppose $f(\bm{x}): R^n \rightarrow R^n$ and $g(\bm{y}): R^n \rightarrow R^n$, where both represent an element-wise binary operation and $|\bm{x}| = |\bm{y}| = n$.

Since input vector space must match that of output, $J(f)$ and $J(g)$ must be `n x n`.

Now, letting $\bm{z}\ = f + g$, we want the Jacobian of $\bm{z}$ w.r.t $\bm{x}$ and $\bm{y}$.

In [8]:
x = np.random.randn(1, 3)
y = np.random.randn(1, 3)

def f(x):
    return x + y

def g(y):
    return x + y

$$J(f) =
\begin{bmatrix}

{\frac{\partial f_1}{\partial x_1}} & {\frac{\partial f_1}{\partial x_2}} & {\frac{\partial f_1}{\partial x_3}} \\
{\frac{\partial f_2}{\partial x_1}} & {\frac{\partial f_2}{\partial x_2}} & {\frac{\partial f_2}{\partial x_3}} \\
{\frac{\partial f_3}{\partial x_1}} & {\frac{\partial f_3}{\partial x_2}} & {\frac{\partial f_3}{\partial x_3}}

\end{bmatrix} = I_3
$$

$$J(g) =
\begin{bmatrix}

{\frac{\partial g_1}{\partial y_1}} & {\frac{\partial g_1}{\partial y_2}} & {\frac{\partial g_1}{\partial y_3}} \\
{\frac{\partial g_2}{\partial y_1}} & {\frac{\partial g_2}{\partial y_2}} & {\frac{\partial g_2}{\partial y_3}} \\
{\frac{\partial g_3}{\partial y_1}} & {\frac{\partial g_3}{\partial y_2}} & {\frac{\partial g_3}{\partial y_3}}

\end{bmatrix} = I_3
$$

The gradient of vector addition for both operands is the identity matrix.

What about the Hadamard product? The idea is similar.

$${x} = \begin{bmatrix} 1 && 2 && 3\end{bmatrix}$$
$${y} = \begin{bmatrix} 4 && 5 && 6\end{bmatrix}$$
$${z} = {x} \odot {y} = \begin{bmatrix} 4 && 10 && 18 \end{bmatrix}$$

$$\frac{\partial {z}}{\partial {x}} = \begin{bmatrix} {\frac{\partial z_1}{\partial x_1}} && {\frac{\partial z_1}{\partial x_2}} && {\frac{\partial z_1}{\partial x_3}} \\
                                                            {\frac{\partial z_2}{\partial x_1}} && {\frac{\partial z_2}{\partial x_2}} && {\frac{\partial z_2}{\partial x_3}} \\
                                                            {\frac{\partial z_3}{\partial x_1}} && {\frac{\partial z_3}{\partial x_2}} && {\frac{\partial z_3}{\partial x_3}} \end{bmatrix} $$

$$  = \begin{bmatrix} {\frac{\partial ({x} \odot {y})_1}{\partial x_1}} && {\frac{\partial ({x} \odot {y})_1}{\partial x_2}} && {\frac{\partial ({x} \odot {y})_1}{\partial x_3}} \\
                                                   {\frac{\partial ({x} \odot {y})_2}{\partial x_1}} && {\frac{\partial ({x} \odot {y})_2}{\partial x_2}} && {\frac{\partial ({x} \odot {y})_2}{\partial x_3}} \\
                                                   {\frac{\partial ({x} \odot {y})_3}{\partial x_1}} && {\frac{\partial ({x} \odot {y})_3}{\partial x_2}} && {\frac{\partial ({x} \odot {y})_3}{\partial x_3}} \end{bmatrix} $$

Again, all diagonal elements are zero'd out, since $\frac{\partial ({x} \odot {y})_n}{\partial x_m} = 0$ when $n \ne m$.

$$ = \begin{bmatrix} 

\frac{\partial ({x} \odot {y})_1}{\partial x_1} && 0 && 0 \\
0 && \frac{\partial ({x} \odot {y})_2}{\partial x_2} && 0 \\
0 && 0 && \frac{\partial ({x} \odot {y})_3}{\partial x_3}

\end{bmatrix} $$

$$ = \begin{bmatrix}
y_1 && 0 && 0 \\
0 && y_2 && 0 \\
0 && 0 && y_3
\end{bmatrix} = diag({y}) $$

The gradient of the Hadamard product is another diagonal matrix. This is good, because we can avoid the cost of matmul in an automatic differentiation framework. We can simply Hadamard product the incoming gradient with the exact vector representation of the other operand, which in this case is `y`.

In [9]:
x = torch.rand(1, 3, requires_grad=True)
y = torch.rand(1, 3, requires_grad=True)
z = x * y
z

tensor([[0.6845, 0.2623, 0.2573]], grad_fn=<MulBackward0>)

In [10]:
z.backward(torch.ones_like(z))
assert (y == x.grad).all()

---

### Vector Sum Gradient

Suppose we have the vector reduction $f: R^n \rightarrow R$, where $f(\bm{x}) = x_1 + x_2 + x_3 + \dots + x_n$.

What would be $J(f)$?
- $J(f)$ would be a linear transformation from $R^n \rightarrow R$ as well, because it is a mapping of how each element of $\bm{x}$ affects $f$.
- In other words, it would be a `1 x n` matrix.

$$\frac{\partial y}{\partial \bm{x}} = \begin{bmatrix}

    \frac{\partial y}{\partial x_1} &&
    \frac{\partial y}{\partial x_2} &&
    \frac{\partial y}{\partial x_3} &&
    \dots && 
    \frac{\partial y}{\partial x_n}
    
    \end{bmatrix}$$ 

$$ = \begin{bmatrix}

    \frac{\partial \sum_{i=1}^{n}{x_i}}{\partial x_1} &&
    \frac{\partial \sum_{i=1}^{n}{x_i}}{\partial x_2} &&
    \frac{\partial \sum_{i=1}^{n}{x_i}}{\partial x_3} &&
    \dots && 
    \frac{\partial \sum_{i=1}^{n}{x_i}}{\partial x_n}
    
    \end{bmatrix}$$ 

$$ = \begin{bmatrix}

    \sum_{i=1}^{n}\frac{\partial {x_i}}{\partial x_1} &&
    \sum_{i=1}^{n}\frac{\partial {x_i}}{\partial x_2} &&
    \sum_{i=1}^{n}\frac{\partial {x_i}}{\partial x_3} &&
    \dots && 
    \sum_{i=1}^{n}\frac{\partial {x_i}}{\partial x_n}
    
    \end{bmatrix}$$ 

The general term $\sum_{i=1}^{n}\frac{\partial {x_i}}{\partial x_n} = 1$, since when $i \ne n$, $\frac{\partial {x_i}}{\partial x_n} = 0$.

Thus, $J(f) = \begin{bmatrix}1 && 1 && 1 && \dots && 1\end{bmatrix}$ regardless of $x_1 \dots x_n$ values.

In [11]:
X = torch.rand(1, 3, requires_grad=True)
Y = X.sum()

In [12]:
Y.backward()
X.grad

tensor([[1., 1., 1.]])

---

### Multivariate Chain Rule

Suppose we have the following chain of operations on vectors.



In [13]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = a + b
e = c.sum()
e

21

$$ \frac{\partial \bm{e}}{\partial \bm{a}} = \frac{\partial \bm{e}}{\partial \bm{c}}\frac{\partial \bm{c}}{\partial \bm{a}} $$

Important to note that the above (simple chain rule) only works because `a` and `b` themselves are not parameterized by another variable. Otherwise we'd need the multivariate law of [total derivatives](https://en.wikipedia.org/wiki/Total_derivative).

Given what we know from above with gradients of elementwise operations & vector summation...

In [14]:
de_dc = np.array([1, 1, 1])
dc_db = np.eye(3)
dc_da = np.eye(3)

In [15]:
de_da = de_dc @ dc_da
de_da

array([1., 1., 1.])

This matmul is pointless; just to show for rigor.

Again, comparing w/ PyTorch...

In [16]:
a = torch.tensor([1., 2., 3.], requires_grad=True)
b = torch.tensor([4., 5., 6.], requires_grad=True)
c = a + b
e = c.sum()
e

tensor(21., grad_fn=<SumBackward0>)

In [17]:
e.backward()
a.grad

tensor([1., 1., 1.])

What about the law of total derivatives mentioned above?

We need this when we need to propagate differentials to output when there are "shared" inputs. For example...

In [18]:
a = torch.tensor([1, 2, 3], requires_grad=True, dtype=torch.float16)
b = a ** 2
c = a + b
d = c.sum()

d.backward()

In [19]:
a.grad

tensor([3., 5., 7.], dtype=torch.float16)

Suppose we want to know $\frac{\partial c}{\partial a}$.

With just knowing the single-variable chain rule, we would attempt to calculate this by computing...

$$ \frac{\partial c}{\partial a} = \frac{\partial c}{\partial b} \frac{\partial b}{\partial a} $$

This would be wrong, however, since $c$ actually depends on $a$ not only (indirectly) from $b$, but also directly from $a$.

$$ a = a $$
$$ b = a^2 $$
$$ c = a + b $$
$$ d = sum(c) $$

We need to sum *all* sources of contributions of $a$ to $c$.

$$ \frac{\partial c}{\partial a} = \frac{\partial c}{\partial a} + \frac{\partial c}{\partial b} \frac{\partial b}{\partial a} $$

To make things clearer, we can denote the $a$ from $a + b$ as $a_1$ and the $a$ from $a^2$ as $a_2$. Then, 

$$ \frac{\partial c}{\partial a} = \frac{\partial c}{\partial a_1} + \frac{\partial c}{\partial b} \frac{\partial b}{\partial a_2} $$

With all of these concepts, we can establish a more computationally efficient framework to compute VJPs--automatic differentiation.

---

### Automatic Differentiation

In [95]:
class AddBackward:
    def __init__(self, child1, child2):
        self.child1 = child1
        self.child2 = child2

    def vjp(self, grad):
        return grad, grad  # since we just matmul the grads with identity matrix, as seen earlier, we can just pass the grad along w/ no modification

In [60]:
class SumBackward:
    def __init__(self, child):
        self.child = child

    def vjp(self, grad):
        return grad * np.ones_like(self.child), None  # effectively useless, but for rigor again

In [75]:
class Value:
    def __init__(self,
                 value: np.array,
                 grad_fn = None,
                 children = (None, None)):
        self.value = value
        self.grad = np.zeros_like(self.value)
        self.grad_fn = grad_fn
        self.children = children
        self.visited = False

    def __repr__(self) -> str:
        return f"Value(value={self.value}, grad_fn={self.grad_fn})"

    def __add__(self, other):
        return Value(value=self.value + other.value,
                     grad_fn=AddBackward(self.value, other.value),
                     children=(self, other))

    def sum(self):
        return Value(value=np.sum(self.value),
                     grad_fn=SumBackward(self.value),
                     children=(self, None))

    def backward(self, grad=None):
        if not (self.value.ndim > 1):  # if we're a scalar value (i.e, f: R^n -> R)
            if grad is None:
                grad = np.array(1)  # kick off gradient chain
        self.grad += grad
        if self.grad_fn:
            vjpL, vjpR = self.grad_fn.vjp(grad)
            if vjpR is None:
                self.children[0].backward(vjpL)
            else:
                self.children[0].backward(vjpL)
                self.children[1].backward(vjpR)

In [76]:
a = Value(np.array([1, 2, 3]))
b = Value(np.array([4, 5, 6]))
c = a + b  # [5, 7, 8]
d = c.sum()

In [77]:
d.backward()

In [78]:
d.grad

array(1)

In [79]:
c.grad

array([1, 1, 1])

In [80]:
a.grad

array([1, 1, 1])

In [81]:
b.grad

array([1, 1, 1])

Nice, everything's as we expect it to be. Let's add support for the Hadamard product.

In [83]:
class MulBackward:
    def __init__(self, child1, child2):
        self.child1 = child1
        self.child2 = child2

    def vjp(self, grad):
        return (grad * self.child2, grad * self.child1)  # from our earlier observation that if z = x * y, the gradient of z w.r.t x = diag(y)

In [84]:
class Value:
    def __init__(self,
                 value: np.array,
                 grad_fn = None,
                 children = (None, None)):
        self.value = value
        self.grad = np.zeros_like(self.value)
        self.grad_fn = grad_fn
        self.children = children

    def __repr__(self) -> str:
        return f"Value(value={self.value}, grad_fn={self.grad_fn})"

    def __add__(self, other):
        return Value(value=self.value + other.value,
                     grad_fn=AddBackward(self.value, other.value),
                     children=(self, other))

    def __mul__(self, other):
        return Value(value=self.value * other.value,
                     grad_fn=MulBackward(self.value, other.value),
                     children=(self, other))

    def sum(self):
        return Value(value=np.sum(self.value),
                     grad_fn=SumBackward(self.value),
                     children=(self, None))

    def backward(self, grad=None):
        if not (self.value.ndim > 1):  # if we're a scalar value (i.e, f: R^n -> R)
            if grad is None:
                grad = np.array(1)  # kick off gradient chain
        self.grad += grad
        if self.grad_fn:
            vjpL, vjpR = self.grad_fn.vjp(grad)
            if vjpR is None:
                self.children[0].backward(vjpL)
            else:
                self.children[0].backward(vjpL)
                self.children[1].backward(vjpR)

Now let's test it on a larger chain of expressions.

In [85]:
a = Value(np.array([1, 2, 3]))
b = Value(np.array([4, 5, 6]))
c = a * b

d = Value(np.array([7, 8, 9]))
e = Value(np.array([10, 11, 12]))
f = d + e

g = c * f
h = g.sum()

In [87]:
h.backward()

In [88]:
h.grad

array(1)

In [89]:
g.grad

array([1, 1, 1])

In [94]:
f.grad, c

(array([ 4, 10, 18]),
 Value(value=[ 4 10 18], grad_fn=<__main__.MulBackward object at 0x13715ba10>))

`f.grad` is what we expect, since it's equal to `c`, which is $[1(4), 2(5), 3(6)]$. Likewise, `c.grad` should be equal to `f`.

In [93]:
c.grad, f

(array([17, 19, 21]),
 Value(value=[17 19 21], grad_fn=<__main__.AddBackward object at 0x1371235f0>))

The big issue with the above implementation is that while it's mathematically correct (at least for the three simple operators seen above), parallelizing these operations properly on a GPU isn't easy.

This automatic differentiation framework effectively creates a large computational DAG, and we need to parallelize a traversal of this graph. We also can't rely on a recursive implementation of passing `grad` around as seen with our current `backward()` method. 

This was the pain point of my earlier [project](https://github.com/Parxd/accelerate), as although I used CuPy as a drop-in replacement for NumPy to try and at least parallelize the pure computations, it couldn't handle a parallel traversal of the actual operation graph, making it very slow and inefficient.

*This* [project](https://github.com/Parxd/cudaML) is an attempt to fix these pain points by using CUDA itself rather than a Python wrapper.

---